In [210]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
import pickle
pickle.dump(cols_input, open('cols_input.sav', 'wb'))
%matplotlib inline
import plotly.offline as py
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [211]:
bank_data = pd.read_csv("bank.csv")

In [212]:
bank_data

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,48,technician,married,professional.course,yes,no,no,cellular,aug,tue,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1,no
1,42,admin.,married,high.school,no,no,no,cellular,nov,mon,...,2,999,1,failure,-0.1,93.200,-42.0,4.191,5195.8,yes
2,71,admin.,married,basic.4y,no,yes,yes,cellular,may,fri,...,4,3,2,success,-1.8,93.876,-40.0,0.684,5008.7,yes
3,39,blue-collar,married,basic.9y,no,yes,no,telephone,oct,fri,...,1,999,0,nonexistent,-0.1,93.798,-40.4,4.918,5195.8,yes
4,67,retired,married,professional.course,no,no,no,telephone,oct,mon,...,2,999,0,nonexistent,-3.4,92.431,-26.9,0.739,5017.5,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4113,37,services,divorced,basic.6y,no,no,no,cellular,nov,thu,...,2,999,0,nonexistent,-0.1,93.200,-42.0,4.076,5195.8,no
4114,45,admin.,married,university.degree,no,no,no,cellular,jul,thu,...,1,999,0,nonexistent,-2.9,92.469,-33.6,1.072,5076.2,yes
4115,60,retired,married,high.school,no,no,no,cellular,sep,wed,...,1,999,0,nonexistent,-1.1,94.199,-37.5,0.876,4963.6,yes
4116,49,blue-collar,married,high.school,unknown,yes,no,cellular,nov,mon,...,1,999,0,nonexistent,-0.1,93.200,-42.0,4.191,5195.8,no


In [213]:
print ('Number of samples: ',len(bank_data))

Number of samples:  4118


In [214]:
bank_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4118 entries, 0 to 4117
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             4118 non-null   int64  
 1   job             4118 non-null   object 
 2   marital         4118 non-null   object 
 3   education       4118 non-null   object 
 4   default         4118 non-null   object 
 5   housing         4118 non-null   object 
 6   loan            4118 non-null   object 
 7   contact         4118 non-null   object 
 8   month           4118 non-null   object 
 9   day_of_week     4118 non-null   object 
 10  duration        4118 non-null   int64  
 11  campaign        4118 non-null   int64  
 12  pdays           4118 non-null   int64  
 13  previous        4118 non-null   int64  
 14  poutcome        4118 non-null   object 
 15  emp.var.rate    4118 non-null   float64
 16  cons.price.idx  4118 non-null   float64
 17  cons.conf.idx   4118 non-null   f

In [215]:
bank_data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,48,technician,married,professional.course,yes,no,no,cellular,aug,tue,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1,no
1,42,admin.,married,high.school,no,no,no,cellular,nov,mon,...,2,999,1,failure,-0.1,93.200,-42.0,4.191,5195.8,yes
2,71,admin.,married,basic.4y,no,yes,yes,cellular,may,fri,...,4,3,2,success,-1.8,93.876,-40.0,0.684,5008.7,yes
3,39,blue-collar,married,basic.9y,no,yes,no,telephone,oct,fri,...,1,999,0,nonexistent,-0.1,93.798,-40.4,4.918,5195.8,yes
4,67,retired,married,professional.course,no,no,no,telephone,oct,mon,...,2,999,0,nonexistent,-3.4,92.431,-26.9,0.739,5017.5,no


In [216]:
bank_data.groupby('y').size()

y
no     2059
yes    2059
dtype: int64

In [217]:
bank_data['OUTPUT'] = (bank_data.y == 'yes').astype('int')

In [218]:
cols_num = ['campaign', 'pdays',
       'previous', 'emp.var.rate', 'cons.price.idx','cons.conf.idx', 'nr.employed','age','euribor3m']

In [219]:
bank_data[cols_num].head()

,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,nr.employed,age,euribor3m
0,1,999,0,1.4,93.444,-36.1,5228.1,48,4.963
1,2,999,1,-0.1,93.200,-42.0,5195.8,42,4.191
2,4,3,2,-1.8,93.876,-40.0,5008.7,71,0.684
3,1,999,0,-0.1,93.798,-40.4,5195.8,39,4.918
4,2,999,0,-3.4,92.431,-26.9,5017.5,67,0.739


In [220]:
cols_cat = ['job', 'marital', 
       'education', 'default',
       'housing', 'loan', 'contact', 'month',
       'day_of_week', 'poutcome']

In [221]:
bank_data[cols_cat].isnull().sum()

job            0
marital        0
education      0
default        0
housing        0
loan           0
contact        0
month          0
day_of_week    0
poutcome       0
dtype: int64

In [222]:
bank_data[cols_cat]
cols_new_cat=pd.get_dummies(bank_data[cols_cat],drop_first = False)
cols_new_cat.head()

,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,...,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,1,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,0,1,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,1,0
4,0,0,0,0,0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0


In [223]:
bank_data = pd.concat([bank_data,cols_new_cat], axis = 1)
cols_all_cat=list(cols_new_cat.columns)
bank_data[cols_all_cat].head()

,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,...,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,1,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,0,1,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,1,0
4,0,0,0,0,0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0


In [225]:
print('Total number of features:', len(cols_all_cat+cols_num))
print('Numerical Features:',len(cols_num))
print('Categorical Features:',len(cols_all_cat))

Total number of features: 62
Numerical Features: 9
Categorical Features: 53


In [226]:
bank_data[cols_num+cols_all_cat].isnull().sum().sort_values(ascending = False)

campaign                 0
month_dec                0
default_no               0
default_unknown          0
default_yes              0
                        ..
education_basic.4y       0
education_basic.6y       0
education_basic.9y       0
education_high.school    0
poutcome_success         0
Length: 62, dtype: int64

In [227]:
cols_input = cols_num + cols_all_cat
bank_data = bank_data[cols_input + ['OUTPUT']]

In [228]:
bank_data.columns

Index(['campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'nr.employed', 'age', 'euribor3m', 'job_admin.',
       'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'default_no', 'default_unknown', 'default_yes',
       'housing_no', 'housing_unknown', 'housing_yes', 'loan_no',
       'loan_unknown', 'loan_yes', 'contact_cellular', 'contact_telephone',
       'month_apr', 'month_aug', 'month_dec', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_s

In [229]:
len(cols_input)

62

In [230]:
bank_data

,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,nr.employed,age,euribor3m,job_admin.,...,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,OUTPUT
0,1,999,0,1.4,93.444,-36.1,5228.1,48,4.963,0,...,0,0,0,0,1,0,0,1,0,0
1,2,999,1,-0.1,93.200,-42.0,5195.8,42,4.191,1,...,0,0,1,0,0,0,1,0,0,1
2,4,3,2,-1.8,93.876,-40.0,5008.7,71,0.684,1,...,0,1,0,0,0,0,0,0,1,1
3,1,999,0,-0.1,93.798,-40.4,5195.8,39,4.918,0,...,0,1,0,0,0,0,0,1,0,1
4,2,999,0,-3.4,92.431,-26.9,5017.5,67,0.739,0,...,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4113,2,999,0,-0.1,93.200,-42.0,5195.8,37,4.076,0,...,0,0,0,1,0,0,0,1,0,0
4114,1,999,0,-2.9,92.469,-33.6,5076.2,45,1.072,1,...,0,0,0,1,0,0,0,1,0,1
4115,1,999,0,-1.1,94.199,-37.5,4963.6,60,0.876,0,...,1,0,0,0,0,1,0,1,0,1
4116,1,999,0,-0.1,93.200,-42.0,5195.8,49,4.191,0,...,0,0,1,0,0,0,0,1,0,0


In [231]:
bank_data = bank_data.sample(n = len(bank_data), random_state = 42)
bank_data = bank_data.reset_index(drop = True)

In [232]:
bank_valid_test=bank_data.sample(frac=0.30,random_state=42)
print('Split size: %.3f'%(len(bank_valid_test)/len(bank_data)))

Split size: 0.300


In [233]:
bank_test = bank_valid_test.sample(frac = 0.5, random_state = 42)
bank_valid = bank_valid_test.drop(bank_test.index)

In [234]:
bank_train_all=bank_data.drop(bank_valid_test.index)

In [235]:
def calc_prevalence(y_actual):
    return (sum(y_actual)/len(y_actual))

In [236]:
print('Test prevalence(n = %d):%.3f'%(len(bank_test),calc_prevalence(bank_test.OUTPUT.values)))
print('Valid prevalence(n = %d):%.3f'%(len(bank_valid),calc_prevalence(bank_valid.OUTPUT.values)))
print('Train all prevalence(n = %d):%.3f'%(len(bank_train_all), calc_prevalence(bank_train_all.OUTPUT.values)))


Test prevalence(n = 618):0.494
Valid prevalence(n = 617):0.506
Train all prevalence(n = 2883):0.500


In [237]:
rows_pos = bank_valid.OUTPUT == 1
bank_valid_pos = bank_valid.loc[rows_pos]
bank_valid_neg = bank_valid.loc[~rows_pos]

bank_valid = pd.concat([bank_valid_pos, bank_valid_neg.sample(n = len(bank_valid_neg), random_state = 42)],axis = 0)

bank_valid = bank_valid.sample(n = len(bank_valid), random_state = 42).reset_index(drop = True)

print('Valid balanced prevalence(n = %d):%.3f'%(len(bank_valid), calc_prevalence(bank_train_all.OUTPUT.values)))


Valid balanced prevalence(n = 617):0.500


In [238]:
rows_pos = bank_test.OUTPUT == 1
bank_test_pos = bank_test.loc[rows_pos]
bank_test_neg = bank_test.loc[~rows_pos]

bank_test = pd.concat([bank_test_pos, bank_test_neg.sample(n = len(bank_test_pos), random_state = 42)],axis = 0)

bank_test = bank_test.sample(n = len(bank_test), random_state = 42).reset_index(drop = True)

print('Test balanced prevalence(n = %d):%.3f'%(len(bank_test), calc_prevalence(bank_train_all.OUTPUT.values)))


Test balanced prevalence(n = 610):0.500


In [239]:
X_train = bank_train_all[cols_input].values
X_train_all = bank_train_all[cols_input].values
X_valid = bank_valid[cols_input].values

y_train = bank_train_all['OUTPUT'].values
y_valid = bank_valid['OUTPUT'].values

print('Training All shapes:',X_train_all.shape)
print('Training shapes:',X_train.shape, y_train.shape)
print('Validation shapes:',X_valid.shape, y_valid.shape)

Training All shapes: (2883, 62)
Training shapes: (2883, 62) (2883,)
Validation shapes: (617, 62) (617,)


In [240]:
scaler  = StandardScaler()
scaler.fit(X_train_all)

StandardScaler()

In [241]:
scalerfile = 'scaler.sav'
pickle.dump(scaler, open(scalerfile, 'wb'))

In [242]:
scaler = pickle.load(open(scalerfile, 'rb'))

In [243]:
X_train_tf = scaler.transform(X_train)
X_valid_tf = scaler.transform(X_valid)

In [244]:
tree = DecisionTreeClassifier(max_depth = 10, random_state = 42)
tree.fit(X_train_tf, y_train)

DecisionTreeClassifier(max_depth=10, random_state=42)

In [253]:
def calc_specificity(y_actual, y_pred, thresh):
    return sum((y_pred < thresh) & (y_actual == 0)) /sum(y_actual ==0)


def print_report(y_actual, y_pred, thresh):
    auc = roc_auc_score(y_actual, y_pred)
    accuracy = accuracy_score(y_actual, (y_pred > thresh))
    recall = recall_score(y_actual, (y_pred > thresh))
    precision = precision_score(y_actual, (y_pred > thresh))
    specificity = calc_specificity(y_actual, y_pred, thresh)
    f1 = 2 * (precision * recall) / (precision + recall)
   
    print('AUC:%.3f'%auc)
    print('accuracy:%.3f'%accuracy)
    print('recall:%.3f'%recall)
    print('precision:%.3f'%precision)
    print('specificity:%.3f'%specificity)
    print('prevalence:%.3f'%calc_prevalence(y_actual))
    print('f1:%.3f'%f1)
    print()
    return auc, accuracy, recall, precision, specificity, f1

In [254]:
thresh = 0.5

In [256]:
y_train_preds = tree.predict_proba(X_train_tf)[:,1]
y_valid_preds = tree.predict_proba(X_valid_tf)[:,1]

print('Decision Tree\n')
print('Training:')
tree_train_auc, tree_train_accuracy, tree_train_recall, tree_train_precision, tree_train_specificity, tree_train_f1 =print_report(y_train,y_train_preds, thresh)
print('Validation:')
tree_valid_auc, tree_valid_accuracy, tree_valid_recall, tree_valid_precision, tree_valid_specificity, tree_valid_f1 = print_report(y_valid,y_valid_preds, thresh)

Decision Tree

Training:
AUC:0.890
accuracy:0.819
recall:0.734
precision:0.884
specificity:0.903
prevalence:0.500
f1:0.802

Validation:
AUC:0.738
accuracy:0.715
recall:0.631
precision:0.764
specificity:0.793
prevalence:0.506
f1:0.691

